In [15]:
import faiss
import numpy as np
import time
import networkx as nx

try:
    res = faiss.StandardGpuResources()
except AttributeError:
    res = None

In [13]:
def load_state(path='./data/states/12340689.txt'):
    with open(path, 'r') as f:
        d = f.read().split("\n")[:-1]
        
    positions, forces, ticker = [], [], 0
    pos, forc = [], []
    for item in d:
        if ticker < 3:
            pos.append(float(item))
        else:
            forc.append(float(item))
        ticker += 1
        if ticker > 5:
            ticker = 0
            positions.append(pos), forces.append(forc)
            pos, forc = [], []
            
    return np.array(positions), np.array(forces)

def build_nn_graph(p, k=6, g=None, use_gpu=True):
    if g is None:
        g = nx.Graph()
#     t1 = time.time()
    p = p.astype("float32")
    index_flat = faiss.IndexFlatL2(3)
    if use_gpu:
        index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
    index_flat.add(p)
    _, I = index_flat.search(p, k+1)
    I = I[:, 1:]
#     print(time.time() - t1)
#     edges = set()
    for i in range(len(p)):
        row = I[i, :]
        for item in row:
            edge = (i, item)
            edge = (min(edge), max(edge))
            if edge[0] != edge[1]:
                g.add_edge(edge[0], edge[1])
    return g

def build_random_graph(p, n_edges_per_node=3, g=None):
    if g is None:
        g = nx.Graph()
    _p = n_edges_per_node/len(p)
    _g = nx.erdos_renyi_graph(len(p), _p)
    g = nx.compose(g, _g)
    return g

def build_throughline_graph(p, g=None):
    if g is None:
        g = nx.Graph()
    last_node = len(p)
    for i in range(last_node):
        other = i+1
        if other == last_node:
            print('reached last node...')
            other = 0
        g.add_edge(i, other)
    return g

def build_graph(p, n_nn_edges=6, n_random_edges=3, do_build_throughline_graph=True, use_gpu=False):
    g = nx.Graph()
    if n_nn_edges > 0:
        g = build_nn_graph(p, k=n_nn_edges, g=g, use_gpu=use_gpu)
    if n_random_edges > 0:
        g = build_random_graph(p, n_edges_per_node=n_random_edges, g=g)
    if do_build_throughline_graph:
        g = build_throughline_graph(p, g)
    return g
    
    
p, f = load_state()
# I = build_nn_graph(p, use_gpu=False)
g = build_graph(p)
print(g.edges())

reached last node...
[(0, 24), (0, 6), (0, 22), (0, 1), (0, 27), (0, 58), (0, 30), (0, 114), (0, 127), (24, 1), (24, 4), (24, 6), (24, 58), (24, 34), (24, 30), (24, 59), (24, 85), (24, 108), (24, 23), (24, 25), (6, 4), (6, 30), (6, 9), (6, 1), (6, 5), (6, 12), (6, 7), (22, 20), (22, 25), (22, 73), (22, 10), (22, 37), (22, 39), (22, 53), (22, 61), (22, 15), (22, 28), (22, 40), (22, 69), (22, 21), (22, 23), (1, 4), (1, 36), (1, 10), (1, 34), (1, 96), (1, 113), (1, 2), (27, 7), (27, 13), (27, 19), (27, 30), (27, 63), (27, 55), (27, 58), (27, 73), (27, 92), (27, 26), (27, 28), (58, 7), (58, 13), (58, 34), (58, 63), (58, 94), (58, 99), (58, 106), (58, 3), (58, 48), (58, 57), (58, 59), (4, 36), (4, 10), (4, 42), (4, 28), (4, 34), (4, 25), (4, 97), (4, 109), (4, 113), (4, 3), (4, 5), (36, 28), (36, 42), (36, 60), (36, 10), (36, 96), (36, 48), (36, 72), (36, 41), (36, 104), (36, 35), (36, 37), (10, 25), (10, 64), (10, 20), (10, 61), (10, 32), (10, 96), (10, 9), (10, 11), (34, 96), (34, 60), (3

In [44]:
p = np.random.random((10000, 3)).astype('float32')

t1 = time.time()
index_flat = faiss.IndexFlatL2(3) 
gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index_flat)
# index = faiss.IndexIVFFlat(3)
gpu_index_flat.add(p)
_, I = gpu_index_flat.search(p, 7)
# print(I[:, 1:])
print(time.time() - t1)

0.006889820098876953


In [2]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.


64

In [20]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])  

True
100000
[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10895 10812 11321]
 [11353 11103 10164  9787]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [17]:
g.edges()

EdgeView([(0, 24), (0, 6), (0, 22), (0, 1), (0, 27), (0, 58), (0, 30), (0, 114), (0, 127), (24, 1), (24, 4), (24, 6), (24, 58), (24, 34), (24, 30), (24, 59), (24, 85), (24, 108), (24, 23), (24, 25), (6, 4), (6, 30), (6, 9), (6, 1), (6, 5), (6, 12), (6, 7), (22, 20), (22, 25), (22, 73), (22, 10), (22, 37), (22, 39), (22, 53), (22, 61), (22, 15), (22, 28), (22, 40), (22, 69), (22, 21), (22, 23), (1, 4), (1, 36), (1, 10), (1, 34), (1, 96), (1, 113), (1, 2), (27, 7), (27, 13), (27, 19), (27, 30), (27, 63), (27, 55), (27, 58), (27, 73), (27, 92), (27, 26), (27, 28), (58, 7), (58, 13), (58, 34), (58, 63), (58, 94), (58, 99), (58, 106), (58, 3), (58, 48), (58, 57), (58, 59), (4, 36), (4, 10), (4, 42), (4, 28), (4, 34), (4, 25), (4, 97), (4, 109), (4, 113), (4, 3), (4, 5), (36, 28), (36, 42), (36, 60), (36, 10), (36, 96), (36, 48), (36, 72), (36, 41), (36, 104), (36, 35), (36, 37), (10, 25), (10, 64), (10, 20), (10, 61), (10, 32), (10, 96), (10, 9), (10, 11), (34, 96), (34, 60), (34, 82), (34,

In [19]:
x = [[40.8340159, -74.00997339999999], [39.7162971, -75.5512786], [40.9019172, -74.11477409999999], [39.7162971, -75.5512786], [40.8371298, -74.0687495], [39.7162971, -75.5512786], [40.8300418, -74.0627834], [40.8371298, -74.0687495], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [39.7162971, -75.5512786], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [40.8300418, -74.0627834], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [37.1498096, 127.0772212], [37.1709185, 127.0435686], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.8300418, -74.0627834], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [42.1026731, -80.1081571], [38.5542324, -121.7647579], [42.4104471, -71.0654462], [-33.8236152, 150.977625], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [37.2613995, 127.0405068], [38.3244002, -121.9619974], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [33.9699488, -118.1580816], [37.4159779, -77.7072238], [37.9703747, -121.7278061], [34.0925424, -117.4796571], [32.6191514, -117.0130506], [40.885219, -73.982429], [34.738301, 135.2149877], [40.2370774, -74.02186449999999], [39.7412903, -75.5879349], [34.03275989999999, -118.4852558], [40.84401200000001, -74.03411729999999], [45.4737692, -122.7762829], [40.84401200000001, -74.03411729999999], [33.7458711, -84.35869869999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [33.7598439, -84.4982146], [37.42802580000001, 126.9960358], [33.6800427, -84.6673026], [33.1938341, -117.3763137], [34.1508149, -118.436892], [39.1588191, -77.2044246], [40.84401200000001, -74.03411729999999], [33.0186832, -80.0341713], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [32.8052778, -96.8426068], [30.4516401, -91.0117454], [33.1261126, -95.53874119999999], [33.6176475, -117.7394219], [40.84401200000001, -74.03411729999999], [39.6076995, -75.7085003], [40.84401200000001, -74.03411729999999], [35.9178135, -78.9236308], [33.7301539, -111.7460485], [39.7491293, -104.3192598], [38.8619762, -76.90670539999999], [33.5280156, -112.1072295], [41.6172987, -72.6558386], [37.9508911, -122.0011365], [45.4737692, -122.7762829], [39.1272878, -76.6858124], [47.5686263, -122.3228778], [25.076936, 121.620375], [14.4958508, 121.0125572], [36.2113714, -86.3019794], [40.4499327, -74.5078987], [43.7223405, -70.4416895], [40.869064, -74.058391], [41.4739917, -81.84299159999999], [33.6269394, -117.8974033], [42.3821693, -71.12171169999999], [38.6084238, -121.2805967], [40.84401200000001, -74.03411729999999], [34.1253447, -118.255803], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [25.9220196, -80.305968], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.6834498, -73.728777], [37.8940581, -122.0649616], [40.4682439, -79.94476949999999], [38.8616309, -77.0627921], [42.0267687, -87.7583848], [32.9006242, -96.8758766], [39.7412903, -75.5879349], [40.5468567, -111.820481], [39.7412903, -75.5879349], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [41.0390683, -73.8154394], [-37.7593929, 144.9606037], [50.8418478, 4.3336587], [14.6420916, 121.1227339], [30.3323554, -89.82932249999999], [42.1244142, -88.25195529999999], [40.84401200000001, -74.03411729999999], [40.9325845, -72.30449550000002], [30.1433554, -95.4791123], [34.6154474, -82.46404249999999], [33.6763151, -117.921376], [39.05437269999999, -76.9356969], [39.7152679, -104.9751199], [33.9262944, -118.4166297], [48.8066944, 2.2959218], [38.51448269999999, -121.4801402], [43.0731209, -89.39360490000001], [-33.7576198, 151.1338796], [47.6687233, -122.2957714], [39.7428804, -75.30182549999999], [40.196373, -85.3945863], [34.0528064, -78.8777841], [38.0027552, -122.5273519], [35.594094, -82.59035829999999], [40.7163883, -73.7534282], [33.5762533, -111.8630913], [41.1500169, -73.2755916], [32.98686120000001, -96.52393409999999], [38.9073649, -76.9937325], [39.0316307, -76.69337639999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [37.71103129999999, -121.8463646], [38.8369387, -94.72755070000001], [36.8066657, -119.8082131], [42.7941686, -86.0322241], [29.7315192, -95.5060915], [39.0546151, -77.0567837], [36.2511833, -115.2485201], [36.0679308, -86.6528238], [33.370189, -84.50841369999999], [38.91829200000001, -76.89352199999999], [47.7268435, -116.8270843], [39.0605619, -76.8417399], [35.8652167, -83.7767633], [40.6847377, -74.2522066], [40.628327, -74.01235179999999], [30.2611423, -95.4359758], [30.3600546, -97.78110620000001], [33.884238, -118.3480554], [32.6532874, -117.0546294], [39.7162971, -75.5512786], [52.54819879999999, 13.3489481], [33.7504656, -84.4738248], [35.1696566, -80.72692040000001], [41.8611315, -71.3962842], [40.0729022, -85.622518], [40.4622623, -79.95072789999999], [32.6085727, -97.3893505], [40.7612505, -74.42525770000002], [41.9737162, -70.7345615], [38.845461, -76.903318], [33.3737898, -111.9083318], [33.7355709, -84.42510349999999], [39.9293112, -75.26830009999999], [29.6275557, -95.41782579999999], [40.84401200000001, -74.03411729999999], [29.9421423, -90.0255986], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [35.1781881, 136.9115881], [41.20433860000001, -73.2011739], [39.1378752, -84.4353392], [38.9931786, -94.6071511], [33.4592202, -82.0654414], [40.7422759, -73.6316598], [52.45173, 13.45104], [40.6690174, -73.56063209999999], [45.0195658, -123.0237385], [29.08261049999999, -81.9863551], [34.0441941, -117.2038092], [46.1262361, -60.196116], [37.53100800000001, 126.9006198], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.7073408, -74.2178867], [41.3428978, -72.92638920000002], [35.6652928, -105.9672107], [44.42808549999999, -93.1738669], [40.84401200000001, -74.03411729999999], [29.949065, -90.10641629999999], [47.79170939999999, -122.3532339], [42.62294199999999, -70.6599445], [43.6567452, -70.2638897], [34.5430107, -112.4755172], [32.8730835, -96.72859799999999], [30.188896, -97.8393663], [1.3643872, 103.8315334], [1.3748425, 103.7399456], [40.84401200000001, -74.03411729999999], [29.6649258, -98.0988955], [40.84401200000001, -74.03411729999999], [41.777763, -71.3932458], [27.6537659, -97.3035285], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [37.7161303, -122.4968061], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.1592185, -111.5801341], [36.1045118, -80.2558817], [47.6505649, -122.3922388], [33.6290901, -112.3511236], [40.1592185, -111.5801341], [47.6246753, -122.2990977], [28.0772692, -82.4679563], [43.51213, -70.384788], [35.9231465, -86.9169608], [32.7055818, -97.0131581], [42.3514811, -71.0433491], [41.7310666, -111.8257953], [40.8589999, -73.9282207], [33.6290901, -112.3511236], [34.1286829, -118.4177527], [25.7390226, -80.2939093], [39.7162971, -75.5512786], [35.23815099999999, 129.0904718], [34.0975709, -118.3192192], [37.7770878, -122.4575477], [40.0043279, -76.6022891], [40.8272147, -73.6982222], [41.9824105, -87.6670326], [40.1592185, -111.5801341], [40.84401200000001, -74.03411729999999], [41.7738184, -73.42177199999999], [37.5803532, -77.4288477], [34.096791, -81.1521161], [-36.8582448, 174.7849424], [40.4281337, -79.91561899999999], [40.885219, -73.982429], [33.7311684, -84.41675359999999], [40.6462248, -73.9502827], [47.49874080000001, -122.3726889], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [34.1260534, -118.0658149], [40.7191692, -74.2426701], [33.7683013, -118.1887403], [35.2464179, -80.83378859999999], [40.84401200000001, -74.03411729999999], [40.4970702, -112.0283315], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [26.7361003, -80.2300619], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [34.099175, -118.286487], [39.7162971, -75.5512786], [42.5933684, -83.2144805], [33.8265086, -117.9382868], [33.8265086, -117.9382868], [38.842622, -76.90358719999999], [42.5954152, -83.2863456], [39.7162971, -75.5512786], [35.3913381, -80.8706629], [40.7645668, -74.0614846], [45.4358775, -122.4928655], [41.2930497, -73.2347762], [41.9837535, -88.1793464], [42.4362581, -79.3269057], [33.8543437, -117.9693306], [40.65723819999999, -74.3784032], [31.4113711, -100.479923], [33.852296, -84.28834119999999], [37.8510788, -122.2793674], [45.5046022, -122.6302128], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [43.6652967, -79.4311925], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.5406187, -111.4112021], [40.8637268, -73.86331489999999], [38.8323606, -76.5346205], [39.7412903, -75.5879349], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [41.870241, -71.3942431], [40.84401200000001, -74.03411729999999], [30.2822916, -97.6471608], [33.5497557, -112.0768163], [36.9416855, -81.0927762], [40.4129463, -75.9220315], [29.2026557, -81.13216880000002], [42.06698129999999, -80.1917565], [40.7855335, -73.9513935], [40.7855335, -73.9513935], [42.5544777, -83.39356540000001], [40.7841244, -73.7092377], [39.7162971, -75.5512786], [35.7895209, -78.6796057], [45.5379272, -122.6257421], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [29.7494798, -95.3679959], [34.0844185, -118.2751189], [40.6729732, -73.9459589], [38.9318814, -77.06838619999999], [42.1605964, -87.9345158], [33.9066626, -117.880924], [40.6789278, -73.9937104], [38.9946616, -94.3818696], [39.7162971, -75.5512786], [51.149407, -114.1888433], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.8496207, -74.07063029999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [38.8558716, -120.0092564], [37.8531611, -122.2617332], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [37.4773388, 126.9556889], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.30558449999999, -76.624089], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [29.4787857, -98.46270589999999], [40.688752, -73.956536], [39.6076995, -75.7085003], [40.6888409, -73.92317], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [47.59756520000001, -122.3336848], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [33.9205602, -118.4137449], [40.84401200000001, -74.03411729999999], [37.2418822, -121.8200737], [25.6294811, -80.4432349], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [42.1106472, -75.9139103], [40.84401200000001, -74.03411729999999], [25.466293, -80.44953], [33.7661394, -118.3331134], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [33.7852736, -84.27756079999999], [40.84401200000001, -74.03411729999999], [37.8430724, -122.2907765], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [34.1081906, -118.3212902], [39.6608175, -75.59165639999999], [57.6979796, 11.9815891], [29.7487481, -91.4057997], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [34.2158774, -118.8486884], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [34.1160556, -118.1926995], [40.84401200000001, -74.03411729999999], [39.6608175, -75.59165639999999], [40.84401200000001, -74.03411729999999], [40.7982764, -73.971662], [39.7412903, -75.5879349], [41.1903897, -73.5854761], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.8371298, -74.0687495], [43.65449700000001, -70.26750799999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.6461457, -73.5345887], [41.9503479, -71.1762008], [40.7202147, -74.0413829], [34.320162, -118.4918249], [39.7412903, -75.5879349], [-37.8702543, 144.9961027], [40.200104, -75.2076571], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [37.7767189, -122.4098882], [29.31166, 47.481766], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [60.16927680000001, 24.9228149], [40.84401200000001, -74.03411729999999], [33.8216523, -118.2694956], [-38.1131144, 147.0688455], [40.84401200000001, -74.03411729999999], [32.1723133, -81.250356], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [42.3472891, -83.0662971], [40.84401200000001, -74.03411729999999], [34.124985, -118.1871207], [33.6672355, -83.98385069999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.6827906, -74.32507149999999], [51.22034679999999, 6.7867485], [29.31166, 47.481766], [47.9187386, -122.1890857], [43.1110649, -89.4797557], [39.9589635, -75.1585443], [38.8607817, -77.09271609999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.92759789999999, -73.86375269999999], [33.4240005, -111.8284692], [30.4112664, -91.03808], [37.5953461, 126.9232987], [37.5953461, 126.9232987], [40.0583238, -74.4056612], [45.6721339, -111.038418], [45.4471643, -122.7863494], [39.7162971, -75.5512786], [39.209591, -84.44131209999999], [-32.9272639, 151.7884228], [40.84401200000001, -74.03411729999999], [39.9339304, -75.1963421], [40.7606254, -73.9903793], [38.9278413, -77.0270977], [28.3601486, -82.5844451], [29.5034308, -98.2805982], [37.6343045, -122.0542028], [37.6385734, -122.0689895], [38.9965488, -76.6819443], [14.6724207, 121.0709452], [30.0815849, -91.9826286], [40.84401200000001, -74.03411729999999], [47.6585526, -122.3450624], [33.7799789, -84.27910829999999], [37.3112082, -121.9508636], [40.84401200000001, -74.03411729999999], [33.6807176, -117.3734631], [37.5953461, 126.9232987], [42.4251037, -83.1463684], [41.7416591, -87.66538039999999], [40.7611279, -73.92407589999999], [39.7412903, -75.5879349], [32.9247894, -97.2686365], [34.0666084, -118.2388355], [41.7363748, -88.27510529999999], [33.8896253, -84.5694473], [39.7412903, -75.5879349], [40.7939617, -74.261605], [38.7559076, -77.0797852], [21.3239958, -157.8557443], [47.6493458, -122.3829431], [40.648337, -73.9689633], [39.97353349999999, -105.2538345], [40.7142299, -73.81519999999999], [42.924636, -78.8469041], [39.7412903, -75.5879349], [32.6982519, -117.0247208], [38.7535258, -77.2739864], [39.9221544, -75.1110368], [33.7746374, -117.9493844], [30.1836487, -95.5820359], [39.6617656, -75.59777179999999], [40.8672358, -73.8947299], [38.7081656, -121.3375659], [40.7515899, -73.8865299], [37.748468, -84.31767549999999], [53.74540349999999, -1.595272], [-27.4815342, 153.0232894], [47.5958072, -122.5659612], [34.0318183, -118.5032062], [34.0804959, -118.268331], [40.84401200000001, -74.03411729999999], [34.0075436, -83.89827410000001], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.8496207, -74.07063029999999], [40.8496207, -74.07063029999999], [40.8496207, -74.07063029999999], [40.8496207, -74.07063029999999], [40.84401200000001, -74.03411729999999], [34.0667304, -118.3941285], [40.9578443, -73.8938649], [27.3030181, -82.5557587], [32.8743891, -96.8117723], [51.5484737, -0.09991939999999999], [34.1561497, -118.5101622], [25.9715362, -80.12629679999999], [41.4900514, -72.8788591], [40.05676, -75.2906962], [40.84401200000001, -74.03411729999999], [33.8842614, -118.3543531], [40.84401200000001, -74.03411729999999], [41.886712, -71.355789], [37.3508963, -122.0681645], [48.89106899999999, 2.3580949], [-34.0346606, 150.7828177], [40.84401200000001, -74.03411729999999], [34.3761727, -89.5197725], [33.1845671, -117.3749137], [33.5437367, -84.0673466], [34.415132, -118.4836819], [40.6610097, -73.9905096], [38.8793228, -76.9839521], [33.5836869, -117.7350007], [41.7781475, -72.7047904], [33.9902284, -118.4722614], [30.3432788, -97.76217079999999], [39.7387304, -104.9767174], [40.84401200000001, -74.03411729999999], [40.8371298, -74.0687495], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [40.8371298, -74.0687495], [38.99820510000001, -77.04104590000001], [47.328217, -122.1809107], [42.273475, -71.5258892], [40.8449838, -73.09708599999999], [37.629829, -97.2063282], [40.769741, -111.8544598], [41.24722860000001, -95.98245209999999], [47.6896105, -122.5275681], [40.84401200000001, -74.03411729999999], [40.8371298, -74.0687495], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [34.2173153, -118.3734502], [39.6608175, -75.59165639999999], [39.7372664, -75.5559308], [41.9353386, -87.6434256], [39.6076995, -75.7085003], [39.9481079, -75.27331269999999], [40.05676, -75.2906962], [29.97153969999999, -95.5075463], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7412903, -75.5879349], [39.7412903, -75.5879349], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [28.3202092, -81.66127329999999], [37.8018357, -122.2323331], [39.9481079, -75.27331269999999], [45.5156644, -122.545652], [33.8751413, -98.5737757], [41.7946304, -87.59696780000002], [41.8985488, -87.67431359999999], [33.9890786, -80.9842798], [36.103523, -115.2938102], [39.6608175, -75.59165639999999], [34.1961539, -118.6562344], [33.4614876, -84.3594421], [41.9114848, -87.676518], [40.84401200000001, -74.03411729999999], [38.3031837, -77.4605399], [36.8675425, -76.2207756], [28.2144074, -81.4762124], [45.5729681, -122.8282625], [43.6594753, -79.3617177], [39.6608175, -75.59165639999999], [50.7025766, 2.8362946], [40.7255613, -73.95205419999999], [42.5916546, -88.4244625], [40.8371298, -74.0687495], [39.6608175, -75.59165639999999], [39.7412903, -75.5879349], [40.84401200000001, -74.03411729999999], [40.8371298, -74.0687495], [39.6608175, -75.59165639999999], [40.8371298, -74.0687495], [38.8270961, -90.4811495], [40.8969437, -73.18822310000002], [45.544687, -122.4727912], [33.1632089, -95.6160007], [37.8270326, -122.2508211], [39.7162971, -75.5512786], [39.2720614, -76.61594269999999], [33.4492471, -117.6485494], [35.0936589, -89.9428205], [39.7162971, -75.5512786], [39.3442409, -76.6204898], [39.7162971, -75.5512786], [29.4837782, -95.1672789], [37.9567278, -121.3078055], [45.5613733, -122.5202263], [34.0824496, -118.3622884], [39.3797094, -76.6188777], [40.6767048, -111.7950492], [39.5378983, -76.8638307], [37.259577, -121.9841231], [40.84401200000001, -74.03411729999999], [39.0063539, -96.86144150000001], [51.5083492, -2.5406047], [22.353249, 114.124065], [40.7515899, -73.8865299], [40.7192783, -73.84170859999999], [40.7188518, -74.0394678], [33.298128, -111.8019833], [36.7547111, -76.002038], [40.1498378, -75.2657025], [33.88798999999999, -118.3879892], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [40.8371298, -74.0687495], [33.1845671, -117.3749137], [36.8148053, -76.1276692], [40.84401200000001, -74.03411729999999], [39.6076995, -75.7085003], [44.5504656, -123.2629279], [40.9974757, -73.6790286], [21.3900886, -157.9222056], [38.8197826, -104.7529827], [33.6512781, -117.7817497], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [29.4809922, -98.411012], [40.84401200000001, -74.03411729999999], [39.6076995, -75.7085003], [21.2834365, -157.8283137], [37.7322645, -122.473194], [37.8012258, -122.2721992], [40.8371298, -74.0687495], [40.8371298, -74.0687495], [40.84401200000001, -74.03411729999999], [33.7548703, -118.1329098], [25.6540871, -80.29036110000001], [32.7471181, -117.0904175], [43.0419218, -87.9067029], [44.9595151, -93.13508379999999], [37.9569314, -122.3709029], [35.2155963, -80.80395109999999], [40.5338886, -74.3128524], [39.7162971, -75.5512786], [33.8227619, -118.0502151], [40.84401200000001, -74.03411729999999], [14.6724207, 121.0709452], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7162971, -75.5512786], [39.7412903, -75.5879349], [39.7162971, -75.5512786], [40.7031566, -111.8122833], [37.3260322, 127.0782523], [40.84401200000001, -74.03411729999999], [39.6608175, -75.59165639999999], [39.6608175, -75.59165639999999], [35.2231842, -80.7832018], [47.5951955, -122.2028188], [33.9016562, -117.7966584], [41.0033096, -74.1549618], [41.1865254, -112.0046244], [37.8486379, -122.2571691], [33.7542949, -118.1332291], [53.6304427, -113.439959], [32.3378406, -90.1772469], [30.0291206, -95.5127921], [37.8486379, -122.2571691], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [39.7162971, -75.5512786], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.885219, -73.982429], [35.040448, -80.7227844], [30.2933682, -97.7299371], [38.8851971, -77.00443], [37.2936263, 127.146836], [40.84401200000001, -74.03411729999999], [40.7248317, -111.8602807], [40.6998915, -73.9136846], [39.6831097, -86.0519108], [40.7488433, -74.2567913], [48.2700788, 16.4527448], [40.7264114, -73.8728632], [33.2300676, -117.345626], [39.5150399, -76.8565559], [46.9288079, 7.4276672], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [39.6608175, -75.59165639999999], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [29.5220722, -98.60612569999999], [41.22179920000001, -85.8171567], [40.8342646, -96.6001344], [14.6420916, 121.1227339], [49.2836698, -123.0906595], [41.91035369999999, -71.1408945], [48.8229936, 2.3727097], [22.335098, 114.140454], [33.6512781, -117.7817497], [40.7226722, -74.0448818], [53.289115, -6.309546500000001], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [43.217498, -77.588754], [42.3549575, -71.0752029], [46.8236707, -96.8938973], [33.8712527, -84.10926789999999], [39.71075, -104.9583341], [41.0461877, -73.5504406], [44.8129973, -93.103537], [40.1567756, -82.89150409999999], [27.0768109, -80.11987119999999], [38.7735068, -89.650954], [37.1994932, 126.8311887], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [35.4699688, -97.5115355], [42.3091847, -71.78182509999999], [38.7735068, -89.650954], [34.0830644, -118.185799], [25.6861076, -80.1706136], [39.6076995, -75.7085003], [45.3135746, -66.0002634], [40.84401200000001, -74.03411729999999], [32.5514604, -85.4325401], [37.4223878, -122.0841877], [45.42438569999999, -122.6787429], [35.9558535, -78.8051457], [37.8107851, -122.2598499], [36.2014381, -81.6642471], [40.84401200000001, -74.03411729999999], [39.518333, -76.634967], [33.7851725, -84.38023539999999], [40.4757869, -79.9374801], [34.1576064, -117.2875658], [34.1576064, -117.2875658], [35.0848893, -106.6035043], [33.8213698, -84.36565139999999], [40.1096492, -88.230676], [33.6512781, -117.7817497], [40.84401200000001, -74.03411729999999], [40.84401200000001, -74.03411729999999], [33.6512781, -117.7817497], [33.6512781, -117.7817497], [40.84401200000001, -74.03411729999999], [40.8300418, -74.0627834], [40.9019172, -74.11477409999999]]

In [20]:
len(x)

1000